In [9]:
import numpy as np
import pandas as pd
import h5py
import json
import os

imports of MISO data

In [135]:
#print(os.getcwd())
seams_transmission_df = pd.read_excel(os.path.join(os.getcwd(),'NREL-Seams Model (MISO).xlsx'),sheet_name='Transmission')
seams_generation_df = pd.read_excel(os.path.join(os.getcwd(),'NREL-Seams Model (MISO).xlsx'),sheet_name='Generation')
seams_load_df = pd.read_excel(os.path.join(os.getcwd(),'NREL-Seams Model (MISO).xlsx'),sheet_name='Load')
seams_mapping_df = pd.read_excel(os.path.join(os.getcwd(),'NREL-Seams Model (MISO).xlsx'),sheet_name='Mapping')

imports of additional data from online MISO-related portals?

checks and formatting of MISO/SEAMS loaded dfs

In [136]:
#clean tx
retain_cols = ['Line','From','To','FW','BW','Area From','Area To']
cleaned_seams_transmission_df = seams_transmission_df[retain_cols].dropna(subset=retain_cols)

#drop generators that aren't in MISO-mapped zones(for now)
#gen_data[gen_data["Unit Type"].isin(self.gentypes)
#seams_generation_df = seams_generation_df[seams_generation_df['Bubble']!='DK'].reset_index()
seams_generation_df = seams_generation_df[seams_generation_df['Bubble'].isin(list(seams_mapping_df['CEP Bus ID']))].reset_index()

notes, if desired

In [144]:
seams_generation_df['Forced Outage Rate']*.01

0      0.024
1      0.024
2      0.024
3      0.024
4      0.024
       ...  
758    0.024
759    0.024
760    0.024
761    0.024
762    0.024
Name: Forced Outage Rate, Length: 763, dtype: float64

In [215]:
### GENERATORS ###
generators_dtype = np.dtype([("name", h5py.special_dtype(vlen=str)), 
                      ("category", h5py.special_dtype(vlen=str)),
                      ("region", h5py.special_dtype(vlen=str))])

generators_data = np.zeros((len(seams_generation_df.index),), dtype=generators_dtype)
generators_data['name'] = tuple(seams_generation_df['Generator Name'])
generators_data['category'] = tuple(seams_generation_df['category'])
generators_data['region'] = tuple(seams_generation_df['Bubble'])

#creates hourly capacity values for all generators
row_len = 8760#5000#len(seams_load_df.index)
capacity_np = np.asarray(np.ones((row_len,1))@np.asmatrix(np.asarray(seams_generation_df['Max Capacity'])),dtype=np.int32)
failure_np = np.asarray(np.ones((row_len,1))@np.asmatrix(np.asarray(seams_generation_df['Forced Outage Rate']*.0001)),dtype=np.float)
repair_np  = np.asarray(np.ones((row_len,len(seams_generation_df.index))),dtype=np.float)*.01#@np.ones(,dtype=np.int32)

### REGIONS ###
regions_dtype = np.dtype([("name",h5py.special_dtype(vlen=str))])
regions_data = np.zeros((len(seams_mapping_df.index),),dtype=regions_dtype)
regions_data['name'] = tuple(seams_mapping_df['CEP Bus ID'])

### INTERFACES ###
interfaces_dtype = np.dtype([("region_from",h5py.special_dtype(vlen=str)),
                         ("region_to",h5py.special_dtype(vlen=str))])

interfaces_data = np.zeros((len(cleaned_seams_transmission_df.index),),dtype=interfaces_dtype)
interfaces_data['region_from'] = tuple(cleaned_seams_transmission_df['From'].astype(int).astype(str))
interfaces_data['region_to'] = tuple(cleaned_seams_transmission_df['To'].astype(int).astype(str))

txfrom_np = np.asarray(np.ones((row_len,1))@np.asmatrix(np.asarray(cleaned_seams_transmission_df['FW'])),dtype=np.int32)
txto_np = np.asarray(np.ones((row_len,1))@np.asmatrix(np.asarray(cleaned_seams_transmission_df['BW'])),dtype=np.int32)

### LINES ###
lines_dtype = np.dtype([("name",h5py.special_dtype(vlen=str)),
                        ("category",h5py.special_dtype(vlen=str)),
                        ("region_from",h5py.special_dtype(vlen=str)),
                         ("region_to",h5py.special_dtype(vlen=str))])

lines_data = np.zeros((len(cleaned_seams_transmission_df.index),),dtype=lines_dtype)
lines_data['name'] = tuple(cleaned_seams_transmission_df['Line'])
lines_data['category'] = tuple(cleaned_seams_transmission_df['Area From'])
lines_data['region_from'] = tuple(cleaned_seams_transmission_df['From'].astype(int).astype(str))
lines_data['region_to'] = tuple(cleaned_seams_transmission_df['To'].astype(int).astype(str))

txfailure_np = np.asarray(np.ones((row_len,len(cleaned_seams_transmission_df.index))),dtype=np.float)*.0001
txrecovery_np = np.asarray(np.ones((row_len,len(cleaned_seams_transmission_df.index))),dtype=np.float)*.01

In [217]:
metadata = {'pras_dataversion':'v0.5.0',
        'start_timestamp':'2012-01-01T00:00:00-05:00',
        'timestep_count': row_len,
                'timestep_length':1,
               'timestep_unit':'h',
               'power_unit':'MW',
               'energy_unit':'MWh'}
#print(len(metadata['start_timestamp']))
with h5py.File('test.pras', 'w',track_order=True) as f:
    #attrs
    for k,v in metadata.items():
        f.attrs[k] = v
        if type(f.attrs[k])==np.int32:
            f.attrs[k] = np.int64(f.attrs[k])#dtype(np.int64)
            print(type(f.attrs[k]))
    #regions
    regions_group = f.create_group('regions')
    regions_group.create_dataset('_core',data=regions_data) #rcore = 
    regions_group.create_dataset('load',data=np.asarray(seams_load_df.iloc[:row_len,1:],dtype=np.int32),dtype=np.int32)
    
    #generators
    generators_group = f.create_group('generators')
    generators_group.create_dataset('_core',data=generators_data) #gcore = 
    generators_group.create_dataset('capacity',data=capacity_np,dtype=np.int32)
    generators_group.create_dataset('failureprobability',data=failure_np,dtype=np.float)
    generators_group.create_dataset('repairprobability',data=repair_np,dtype=np.float)
    
    #interfaces
    interfaces_group = f.create_group('interfaces')
    interfaces_group.create_dataset('_core',data=interfaces_data)
    interfaces_group.create_dataset('forwardcapacity',data=txfrom_np,dtype=np.int32)
    interfaces_group.create_dataset('backwardcapacity',data=txto_np,dtype=np.int32)
    
    
    #lines
    lines_group = f.create_group('lines')
    lines_group.create_dataset('_core',data=lines_data)
    lines_group.create_dataset('forwardcapacity',data=txfrom_np,dtype=np.int32)
    lines_group.create_dataset('backwardcapacity',data=txto_np,dtype=np.int32)
    lines_group.create_dataset('failureprobability',data=txfailure_np,dtype=np.float)
    lines_group.create_dataset('repairprobability',data=txrecovery_np,dtype=np.float)
    
    #dset_int_1 = f.create_dataset('integers', (10, ), dtype='i1')
    #d = g.create_dataset('default', data=arr)
    #generators_index = generators_group.create_dataset('_core',data=np.asarray(seams_mapping_df['CEP Bus ID']))#,dtype='Compound'

<class 'numpy.int64'>
<class 'numpy.int64'>


In [84]:
'''
with h5py.File('test.hdf5') as f:
    #dset = f.regions_group['_core']
    print(rcore)
    #print("%s:" % DATASET)
    print("\tSerial number:  {0}".format(dset["name"]))
    #print("\tLocation:  {0}".format(rdata["Location"]))
    #print("\tTemperature:  {0}".format(rdata["Temperature"]))
    #print("\tPressure:  {0}".format(rdata["Pressure"]))
'''
with h5py.File('test.hdf5', "r") as f:
    # List all groups
    #print(f.values)
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = f[a_group_key]
    print(data)
    print(type(data))

Keys: <KeysViewHDF5 ['generators', 'regions']>
<HDF5 group "/generators" (1 members)>
<class 'h5py._hl.group.Group'>


In [ ]:
#https://support.hdfgroup.org/ftp/HDF5/examples/python/hdf5examples-py/high_level/h5ex_t_cmpd.py
"""
This example shows how to read and write compound datatypes to a dataset.  The
program first writes compound structures to a dataset with a dataspace of DIM0,
then closes the file.  Next, it reopens the file, reads back the data,
and outputs it to the screen.
"""
import numpy as np
import h5py

FILE = "h5ex_t_cmpd.h5"
DATASET = "DS1"

DIM0 = 4

def run():

    # Create the compound datatype.
    dtype = np.dtype([("Serial number", np.int32), 
                      ("Location",      h5py.special_dtype(vlen=str)),
                      ("Temperature",   np.float),
                      ("Pressure",      np.float)])

    wdata = np.zeros((DIM0,), dtype=dtype)
    wdata['Serial number'] = (1153, 1184, 1027, 1313)
    wdata['Location'] = ("Exterior (static)", "Intake", "Intake manifold",
                         "Exhaust manifold")
    wdata['Temperature'] = (53.23, 55.12, 103.55, 1252.89)
    wdata['Pressure'] = (24.57, 22.95, 31.23, 84.11)

    with h5py.File(FILE, 'w') as f:
        dset = f.create_dataset(DATASET, (DIM0,), dtype=dtype)
        dset[...] = wdata


    with h5py.File(FILE) as f:
        dset = f[DATASET]
        rdata = dset[...]

    print("%s:" % DATASET)
    print("\tSerial number:  {0}".format(rdata["Serial number"]))
    print("\tLocation:  {0}".format(rdata["Location"]))
    print("\tTemperature:  {0}".format(rdata["Temperature"]))
    print("\tPressure:  {0}".format(rdata["Pressure"]))


if __name__ == "__main__":
    run()        
   
